<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!python3 main.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 25.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=ac6c810fe973a6539d754b3b96a6bc733cf20d3a0f33405dd77204d3a49566b7
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 456 kB 34.9 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460123 sha256=e9ce4c84483cc71b648c5ad974be721233fff225dafeb28ac0cc7bd197ae3cb0
  Stored in directory: /root/.cache/pip/wheels/f2/a3/85/fe8b65a65a447c9906e3b7edb7d9e6c74dfa9c8425c3dd3007
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3943 sha256=ca79388ebd48ab5f6793c388d6038431e27f

In [7]:
from pyngrok import ngrok

import os  
import shutil  


from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000

import sqlite3 as sql


con = sql.connect("/content/photogrammetry_meshroom_colab_flask/sezerdb.db", check_same_thread=False)
cursor = con.cursor()

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys

import random


In [56]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/photogrammetry_meshroom_colab_flask/templates"
app = Flask(__name__, template_folder=TEMPLATE)
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
islem_durumu=False;


app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/")
def main_page():

    print("Anasayfaya donuldu")
    return render_template("HomePage.html")

@app.route("/uploadimages", methods=["GET", "POST"])
def upload_file():
    print("resimler secildi")
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            print("Görseller kaydedildi")
            return redirect(url_for('Show_Process'))
    return render_template("ShowSelectedImages.html")
  
@app.route("/process")
def Show_Process():
  if(islem_durumu==False):
    return redirect(url_for('Process'))
  return render_template("processing.html")
  print("isleniyor")

  
@app.route("/sezer")
def Process():
  return render_template("showModel.html")
  return "çalisti"
  os.system("rembg p /content/kirli   /content/temiz")
  os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input /content/temiz --output /content/modelFolder")
  os.system("obj2gltf -i /content/modelFolder/model.obj -o /content/static/model.glb")
  
@app.route("/SingUp", methods=["GET", "POST"])
def SingUp():
   if request.method == 'POST':
        name = request.form.get("name")
        surname = request.form.get("surname")
        global email
      
        email = request.form.get("email")
        password = request.form.get("password")
        print("before db")
        print(name)
        #cursor.execute("INSERT INTO Users VALUES(name,surname,password,email)")
        try:
          cursor.execute("insert into Users values(?,?,?,?,?,?)",(name, surname, password,email,0,None))
          con.commit()
          print("Yeni Kullanıcı kaydedildi")
        except:
            print("Kaydedilmedi")
        try:
          global user_id
          cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (email, password))
          veri = cursor.fetchall()
          user_id= veri[0][0]
          print(veri[0][0])
          print("Kullanıcı Id alındı")
        except:
            print("Id alınamadi")   
        # Gmail email sunucusuna bağlanıyoruz
        try:
            global ver_code
            ver_code = random.randint(100000,999999)
            print("vercode", ver_code)
            mail = smtplib.SMTP("smtp.gmail.com",587)
            mail.ehlo()
            mail.starttls()
            mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")
            mesaj = MIMEMultipart()
            mesaj["From"] = "3D AR Object Platform "           # Gönderen
            mesaj["Subject"] = "Verification Code"   # Konusu
            mesaj["To"] = email
            body = "Your Verification Code :  "+ str(ver_code) 
            body_text = MIMEText(body, "plain")  #
            mesaj.attach(body_text)
            mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
            print("Mail başarılı bir şekilde gönderildi.")
            mail.close()
            return render_template("SentVerificationCode.html")
        # Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
        except:
            print("Hata:", sys.exc_info()[0])
            
   return render_template("SingUp.html")

@app.route("/SentCode", methods=["GET", "POST"])
def SentCode():
  if request.method == 'POST':
    code = request.form.get("sifre")
    print("gelen kod", ver_code)
    print("girilen kod", code)

    if(int(code) == ver_code):
      cursor.execute("UPDATE Users SET Is_Active = 1 WHERE Email = email" )
      con.commit()
      return render_template("Login.html",yazi="Tebrikler mailiniz onaylandı şimdi giriş yapabilirsiniz.")
    else:
      return render_template("SentVerificationCode.html",yazi="Mail onaylanmadı kodu tekrar giriniz.")
  return render_template("SentVerificationCode.html")

  
@app.route("/CheckLogin", methods=["GET", "POST"])
def CheckLogin():
  if request.method == 'POST':
    user_mail = request.form.get("email")
    user_password = request.form.get("password")
    print("Check",user_mail)
    print("Check2",user_password)
    cursor.execute("SELECT * FROM  Users WHERE Email = ? and Password = ? ", (user_mail, user_password))
    veri = cursor.fetchall() 
    if len(veri) != 0:
      return render_template("AddProduct.html")
    else:
      return render_template("Login.html",yazi="Kullanıcı maili veya şifre yanlış tekrar deneyiniz")
  
@app.route("/SaveProduct", methods=["GET", "POST"])
def SaveProduct():
  if request.method == 'POST':
    product_name = request.form.get("product_name")
    product_info = request.form.get("product_info")
    user_place1 = request.form.get("place1")
    user_place2= request.form.get("place2")
    user_place3 = request.form.get("place3")

    cursor.execute("insert into Products values(?,?,?,?,?,?,?,?)",(product_name,user_id,None, product_info, None,user_place1,user_place2,user_place3))
    con.commit()
    
    return render_template("Profil.html",isim=product_name,bilgi=product_info,ozel_url=None, url1=user_place1,url2=user_place2,url3=user_place3)
   
      

       
if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://d6f0-34-143-133-250.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:49:11] "GET / HTTP/1.1" 200 -


Anasayfaya donuldu


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:49:14] "GET /playground_assets/logo-200h.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:49:18] "GET /uploadimages HTTP/1.1" 200 -


resimler secildi


...

INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:50:07] "POST /SentCode HTTP/1.1" 200 -


gelen kod 407412
girilen kod 407412


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:50:22] "POST /CheckLogin HTTP/1.1" 200 -


Check sezerbugday2@gmail.com
Check2 0000


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:50:49] "POST /SaveProduct HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:50:51] "GET /node_modules/xterm/css/xterm.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:50:51] "GET /node_modules/xterm/lib/xterm.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 21:50:51] "GET /static/kup.glb HTTP/1.1" 404 -


In [ ]:


# Gmail email sunucusuna bağlanıyoruz
try:
    mail = smtplib.SMTP("smtp.gmail.com",587)
    mail.ehlo()
    mail.starttls()
    mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")

    mesaj = MIMEMultipart()
    mesaj["From"] = "3D AR Object Platform "           # Gönderen
    mesaj["Subject"] = "Verification Code"    # Konusu
    mesaj["To"] = "sezerbugday2@gmail.com "
    body = " Bu mesaj sezer tarafından python ile gönderilmiştir "

    body_text = MIMEText(body, "plain")  #
    mesaj.attach(body_text)

    mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
    print("Mail başarılı bir şekilde gönderildi.")
    mail.close()

# Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
except:
    print("Hata:", sys.exc_info()[0])

Mail başarılı bir şekilde gönderildi.


In [50]:
cursor.execute("Select * From Products")
veri = cursor.fetchall() 
print(veri)
cursor.execute("Select * From Users")
user_veri = cursor.fetchall() 
print(user_veri)

[('urun', 55, 6, 'urun info', None, 'google', 'trendyol', 'facebook')]
[('sezer', 'bugday', '123456', 'sezerbugday2@gmail.com', 1, 55)]


In [43]:
password="123456"
cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (email, password))
veri = cursor.fetchall()
print(veri[0][0])

52


In [ ]:
cursor.execute("insert into Users values(?,?,?,?,?,?)",("name", "surname", "123456","sezerbugday2@gmail.com",0,None))
con.commit()

In [23]:
cursor.execute("SELECT * FROM  Users WHERE Email = 'sezerbugday2@gmail.com' and Password = 123456")
veri = cursor.fetchall() 
print(veri)
print(len(veri))

[('name', 'surname', '123456', 'sezerbugday2@gmail.com', 1, 35), ('dsfd', 'sdfdsf', '123456', 'sezerbugday2@gmail.com', 1, 40), ('fsddfs', 'sdfsdfds', '123456', 'sezerbugday2@gmail.com', 1, 43)]
3


In [47]:
def VeriSil():
    cursor.execute("DELETE FROM Products")
    con.commit()
VeriSil()

In [54]:
cursor.execute("SELECT Products.CustomerID,Users.name,Users.Surname,Products.Name, Products.ProductID FROM Users INNER JOIN Products ON Users.CustomerID = Products.CustomerID ;")
veri = cursor.fetchall() 
print(veri)


[(55, 'sezer', 'bugday', 'urun', 6)]


In [21]:
cursor.execute("SELECT * From Products")
veri = cursor.fetchall() 
print(veri)

[]


In [28]:
user_mail = 'sezerbugday2@gmail.com'
user_password= '123456'
cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (user_mail, user_password))
veri = cursor.fetchall() 
print(veri[1][0])

40


In [57]:
%cd /content/photogrammetry_meshroom_colab_flask

/content/photogrammetry_meshroom_colab_flask


In [58]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   sezerdb.db
	modified:   templates/AddProduct.html
	modified:   templates/Profil.html

no changes added to commit (use "git add" and/or "git commit -a")


In [59]:
!git add --all  

In [61]:
!git config --global user.email "sezerbugday2@gmail.com"

In [62]:
!git commit -a -m "urun tablosu eklendi"


[main 8f0acd0] urun tablosu eklendi
 3 files changed, 24 insertions(+), 9 deletions(-)


In [63]:
!git branch

* main


In [64]:
!git push origin main

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.24 KiB | 1.24 MiB/s, done.
Total 6 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/SezerBugday/photogrammetry_meshroom_colab_flask.git
   5c6afc6..8f0acd0  main -> main
